In [2]:
path = '/home/raza.imam/Documents/HC701B/Project/data/TB_data/testing/Normal/Normal-4.png'
import cv2

# Load the image
img = cv2.imread(path)

# Get the shape of the image
height, width, channels = img.shape

print(f"Image shape: {height} x {width} x {channels}")


Image shape: 512 x 512 x 3


In [1]:
import os 
import torch 
import argparse
import numpy as np

from utils import *
from data.dataset import data_loader, data_loader_attacks
from tqdm import tqdm

import mlp

def majority_voting(data_loader, model, mlps_list):
    """
    SEViT performance with majority voting. 

    Args: 
    data_loader: loader of test samples for clean images, or attackes generated from the test samples
    model: ViT model 
    mlps_list: list of intermediate MLPs

    Return: 
    Accuracy. 
    """

    acc_ = 0.0 
    for images, labels in tqdm(data_loader):
        final_prediction = []
        images = images.cuda()
        vit_output = model(images)
        vit_predictions = torch.argmax(vit_output.detach().cpu(), dim=-1)
        final_prediction.append(vit_predictions.detach().cpu())
        x = model.patch_embed(images)
        x_0 = model.pos_drop(x)
        i=0
        for mlp in mlps_list:
            x_0 = model.blocks[i](x_0)
            mlp_output = mlp(x_0)
            mlp_predictions = torch.argmax(mlp_output.detach().cpu(), dim=-1)
            final_prediction.append(mlp_predictions.detach().cpu())
            i+=1
        stacked_tesnor = torch.stack(final_prediction,dim=1)
        preds_major = torch.argmax(torch.nn.functional.one_hot(stacked_tesnor).sum(dim=1), dim=-1)
        acc = (preds_major == labels).sum().item()/len(labels)
        acc_ += acc
    final_acc = acc_ / len(data_loader)
    print(f'Final Accuracy From Majority Voting = {(final_acc *100) :.3f}%' )
    return final_acc

def countParams(model):
    total_params = sum(param.numel() for param in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Total Params: ", total_params)
    print("Trainable Params: ", trainable_params)

def vote(images_type, mlp_path):
    root_dir = 'data/TB_data/'
    vit_path = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
    # mlp_path = 'models/MLP_new_chest/'

    model = torch.load(vit_path).cuda()
    model.eval()
    print('ViT is loaded!')

    MLPs_list = get_classifiers_list(MLP_path=mlp_path, num_classifiers=5)
    print('All MLPs are loaded!')

    if images_type == 'clean':
        image_folder_path = 'data/TB_data'
        loader_, dataset_ = data_loader(root_dir=image_folder_path, batch_size=15)
        majority_voting(data_loader=loader_['test'], model= model, mlps_list=MLPs_list)
        # average(data_loader=loader_['test'], model= model, mlps_list=MLPs_list)
        # weighted_average(data_loader=loader_['test'], model= model, mlps_list=MLPs_list)
    elif images_type == 'attack':
        image_folder_path = 'attack_images'
        attack_name = 'AUTOPGD'
        loader_, dataset_ = data_loader_attacks(root_dir=image_folder_path, attack_name= attack_name, batch_size=15)
        majority_voting(data_loader=loader_, model= model, mlps_list=MLPs_list)
        # average(data_loader=loader_, model= model, mlps_list=MLPs_list)
        # weighted_average(data_loader=loader_, model= model, mlps_list=MLPs_list)
    else:
        print('Input correct image type')

# path = 'ReVIT/models/cnns/'
# vote('attack', path)

Before adversaring Training. Table 1-3

In [ ]:
# Table 1: Clean Performance of MLP alternatives, from m=1 to m=5

""" def Table1(images_type, mlp_path):
    root_dir = 'data/TB_data/'
    vit_path = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

    model = torch.load(vit_path).cuda()
    model.eval()
    print('ViT is loaded!')

    MLPs_list = get_classifiers_list(MLP_path=mlp_path, num_classifiers=5)
    print('All MLPs are loaded!')
    print("# of parameters of a module:")
    countParams(MLPs_list[0])

    if images_type == 'clean':
        image_folder_path = 'data/TB_data'
        loader_, dataset_ = data_loader(root_dir=image_folder_path, batch_size=15)
        for i in range(0, 6): # 0=VIT, 1 to 5 = mlp modules
            MLPs_subset = MLPs_list[:i]
            print(len(MLPs_subset))
            print(f"For m = {i}")        
            majority_voting(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
            average(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
            weighted_average(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
    elif images_type == 'attack':
        image_folder_path = 'attack_images'
        attack_name = 'AUTOPGD'
        loader_, dataset_ = data_loader_attacks(root_dir=image_folder_path, attack_name= attack_name, batch_size=15)
        majority_voting(data_loader=loader_, model= model, mlps_list=MLPs_list)
        average(data_loader=loader_, model= model, mlps_list=MLPs_list)
        weighted_average(data_loader=loader_, model= model, mlps_list=MLPs_list)
    else:
        print('Input correct image type')

path_list = ['ReVIT/models/cnns', 'ReVIT/models/resnets_ft', 'ReVIT/models/resnets_tl', 'ReVIT/models/resnets_ft_cnn', 'ReVIT/models/resnets_tl_cnn', 'ReVIT/models/mlps']
for i in range(0, 6):
    path = path_list[i]
    print("For", path.split('/')[-1])
    Table1('clean', path)

# MLPs_list = get_classifiers_list(MLP_path=path, num_classifiers=5)
# print(MLPs_list)
# print('All MLPs are loaded!')
"""

# __Testing__ Diffusion samples on SEVIT of MLP ensmbles

In [4]:
# Table X: Attack Performance (Diffusion Samples) of MLP ensembles, from m=1 to m=4
import six
def Table2(images_type, mlp_path):
    if images_type == 'clean':
        diff_dir = 'data/Diffusion_TB_data/sanity_check_blr'
        image_folder_path = diff_dir
        loader_, dataset_ = data_loader(root_dir=image_folder_path, batch_size=15)
        
        vit_path = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'
        model = torch.load(vit_path).cuda()
        model.eval()
        print('ViT is loaded!')

        MLPs_list = get_classifiers_list(MLP_path=mlp_path, num_classifiers=4)
        print('All MLPs are loaded!')
        print("# of parameters of a module:")
        countParams(MLPs_list[0])
    
        for i in range(0, 6): # 0=VIT, 1 to 5 = mlp modules
            MLPs_subset = MLPs_list[:i]
            print(len(MLPs_subset))
            print(f"For m = {i}")        
            majority_voting(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
            # average(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
            # weighted_average(data_loader=loader_['test'], model= model, mlps_list=MLPs_subset)
    else:
        print('Input correct image type')


path = 'models/MLP_new_chest'
print("\n\n-->>> For", path.split('/')[-1])
Table2('clean', path)

# MLPs_list = get_classifiers_list(MLP_path=path, num_classifiers=5)
# print(MLPs_list)
# print('All MLPs are loaded!')



-->>> For MLP_new_chest
['number of train images is 2', 'number of valid images is 2', 'number of test images is 698']
Classes with index are: {'Normal': 0, 'Tuberculosis': 1}
['Normal', 'Tuberculosis']
ViT is loaded!
mlp_block_0_classifier_0.94test_0.98train.pth
MLP 1 is loaded!
mlp_block_1_classifier_0.93test_0.99train.pth
MLP 2 is loaded!
mlp_block_2_classifier_0.94test_0.99train.pth
MLP 3 is loaded!
mlp_block_3_classifier_0.93test_0.99train.pth
MLP 4 is loaded!
mlp_block_4_classifier_0.92test_1.00train.pth
All MLPs are loaded!
# of parameters of a module:
Total Params:  625219970
Trainable Params:  625219970
0
For m = 0


  0%|          | 0/46 [00:00<?, ?it/s]


AttributeError: 'VisionTransformer' object has no attribute 'no_embed_class'

: 

In [ ]:
# Table 2.1: Attack Performance (on adversarial examples generated for the surrogate model) of MLP alternatives, from m=1 to m=4
# Perturbation Budget is 0.03
"""
def Table2_1(images_type, mlp_path):
    # root_dir = 'data/TB_data/'
    vit_path = 'models/vit_base_patch16_224_in21k_test-accuracy_0.96_chest.pth'

    model = torch.load(vit_path).cuda()
    model.eval()
    print('ViT is loaded!')

    MLPs_list = get_classifiers_list(MLP_path=mlp_path, num_classifiers=4)
    print('All MLPs are loaded!')
    print("# of parameters of a module:")
    countParams(MLPs_list[0])

    if images_type == 'attack':
        image_folder_path = 'attack_images_surrogate_0.03'
        attack_list = ['FGSM', 'PGD', 'BIM', 'AUTOPGD', 'CW']
        for i in attack_list: #For all attack types in attack_list
            attack_name = i
            print(f'\nFor {attack_name} Attack samples:')
            loader_, dataset_ = data_loader_attacks(root_dir=image_folder_path, attack_name= attack_name, batch_size=15)
            for i in range(1, 5):
                MLPs_subset = MLPs_list[:i]
                print(len(MLPs_subset))
                print(f"For m = {i}")        
                majority_voting(data_loader=loader_, model= model, mlps_list=MLPs_subset)
                average(data_loader=loader_, model= model, mlps_list=MLPs_subset)
                weighted_average(data_loader=loader_, model= model, mlps_list=MLPs_subset)
    else:
        print('Input correct image type')

path_list = ['ReVIT/models/cnns', 'ReVIT/models/resnets_ft', 'ReVIT/models/resnets_tl', 'ReVIT/models/resnets_ft_cnn', 'ReVIT/models/resnets_tl_cnn', 'ReVIT/models/mlps']
for i in range(0, 6):
    path = path_list[i]
    print("\n\n-->>> For", path.split('/')[-1])
    Table2_1('attack', path)

# MLPs_list = get_classifiers_list(MLP_path=path, num_classifiers=5)
# print(MLPs_list)
# print('All MLPs are loaded!')
"""